In [ ]:
import os
import glob
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#brain packages
import nltools
from nltools import Brain_Data
import nibabel as nb
import nilearn
from nilearn import plotting
from nilearn.image import resample_to_img, math_img
from nilearn.mass_univariate import permuted_ols
from nilearn.glm.second_level import non_parametric_inference

In [ ]:
isc_outs = '/dartfs/rc/lab/S/Scraplab/psypose_fmri/isc_analysis/'

tasknames = ['pulpfiction','500daysofsummer','backtothefuture','citizenfour',
           'littlemisssunshine', '12yearsaslave','split','theprestige',
           'theshawshankredemption','theusualsuspects']

betalist = []
for task in tasknames:
    betalist.append(isc_outs+task+os.sep+task+"_betamap.nii.gz")

hypotheses = ["zero_vs_one","one_vs_two","two_vs_three","three_vs_four+",
              "one_vs_four+","scd","speech","overlap","arousal","valence"]
a,b = 0,1

# Running participant-wise permutations for each hypothesis

Each participant has an image object containing beta images corresponding to the various regressors we modeled their functional timeseres data with. Here, we load in that image object, select the specific beta images we want to conduct a one-sample t-test on (or, select two beta images if we're looking to run a one-sample t-test on a contrasted beta map), and submit a list of these images for all 10 movies to our permutation analysis.

In [ ]:
for i,test in enumerate(hypotheses):
    if i == slurm_input: #allows for parallelization
        inputs = []
        if i <= 4:
            if i==0:
                a,b=0,1
            if i == 1:
                a,b, = 1,2
            if i == 2:
                a,b = 2,3
            if i == 3:
                a,b = 3,4
            if i == 4:
                a,b,=1,4
            
            for image in betalist:
                data = nb.load(image)
                inputs.append(math_img("img1 - img2", img1=data.slicer[:,:,:,a], img2=data.slicer[:,:,:,b]))
            
            dm = pd.DataFrame([1] * len(inputs),columns=['intercept'])
            
            out_dict = nilearn.glm.second_level.non_parametric_inference(inputs, confounds=None, design_matrix=dm,
                                                      second_level_contrast=None, first_level_contrast=None, mask=None,
                                                      smoothing_fwhm=None, model_intercept=True, n_perm=5000,
                                                      two_sided_test=True, random_state=None, n_jobs=1, verbose=1,
                                                      threshold=None, tfce=True)
            for item in out_dict:
                key = out_dict[item]
                nb.save(key,isc_outs+"permutation_outs/"+item+"_"+test+"_.nii.gz")
            
        else:
            for image in betalist:
                data = nb.load(image)
                inputs.append(data.slicer[:,:,:,i])
            
            dm = pd.DataFrame([1] * len(inputs),columns=['intercept'])
            
            out_dict = nilearn.glm.second_level.non_parametric_inference(inputs, confounds=None, design_matrix=dm,
                                                      second_level_contrast=None, first_level_contrast=None, mask=None,
                                                      smoothing_fwhm=None, model_intercept=True, n_perm=5000,
                                                      two_sided_test=True, random_state=None, n_jobs=1, verbose=1,
                                                      threshold=None, tfce=True)
            for item in out_dict:
                key = out_dict[item]
                nb.save(key,isc_outs+"permutation_outs/"+item+"_"+test+"_.nii.gz")
    else:
        continue